In [13]:

# find model from record of qlib
%load_ext autoreload
%autoreload 2
import sys
print(sys.path)
sys.path.insert(0, "/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib")

import qlib
print("qlib:",qlib.__path__)

qlib.init()

[33612:MainThread](2023-11-21 11:16:59,900) INFO - qlib.Initialization - [config.py:417] - default_conf: client.
[33612:MainThread](2023-11-21 11:16:59,902) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[33612:MainThread](2023-11-21 11:16:59,903) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/home/codespace/.qlib/qlib_data/cn_data')}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib', '/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib', '/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib', '/workspaces/Portfolio-Leveraging-Differentiable-Layer/experiment', '/opt/conda/envs/qlib/lib/python38.zip', '/opt/conda/envs/qlib/lib/python3.8', '/opt/conda/envs/qlib/lib/python3.8/lib-dynload', '', '/opt/conda/envs/qlib/lib/python3.8/site-packages', '/opt/conda/envs/qlib/lib/python3.8/site-packages/pyqlib-0.9.1.99-py3.8-linux-x86_64.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/protobuf-3.20.1-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/gym-0.26.2-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/Jinja2-3.0.3-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/filelock-3.13.1-py3.8.egg', '/opt/conda/envs/qlib/lib/python3.8/site-packages/dill-0.3.7-py3.8.egg', '/op

## load the experiment you conducted before 

In [14]:
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord, SigAnaRecord
R.list_experiments()

{'ExpQuickStartGRU': MLflowExperiment(id=466892454790696298, info={'class': 'Experiment', 'id': '466892454790696298', 'name': 'ExpQuickStartGRU', 'active_recorder': None, 'recorders': ['05eee8f1acc94d1280152e884d344dad']}),
 'Default': MLflowExperiment(id=0, info={'class': 'Experiment', 'id': '0', 'name': 'Default', 'active_recorder': None, 'recorders': []})}

## Find the recorder that corresponding to your prediction result/model

In [15]:
EXP_NAME ="ExpQuickStartGRU"
rid = "05eee8f1acc94d1280152e884d344dad"
rec = R.get_recorder(experiment_name=EXP_NAME,recorder_id=rid)

### load prediction result from your record

In [16]:
pred = rec.load_object("pred.pkl")

### run backtest based on pred

In [18]:
port_analysis_config2 = {
    "executor": {
        "class": "SimulatorExecutor",
        "module_path": "qlib.backtest.executor",
        "kwargs": {
            "time_per_step": "day",
            "generate_portfolio_metrics": True,
        },
    },
    "strategy": {
        "class": "TopkDropoutStrategy",
        "module_path": "qlib.contrib.strategy.signal_strategy",
        "kwargs": {
            "signal": "<PRED>",
            "topk": 50,
            "n_drop": 50,
        },
    },
    "backtest": {
        "start_time": "2019-01-01",
        "end_time": "2019-12-31",
        "account": 100000000,
        "benchmark": "SH000300",
        "exchange_kwargs": {
            "freq": "day",
            "limit_threshold": 0.095,
            "deal_price": "close",
            "open_cost": 0.0005,
            "close_cost": 0.0015,
            "min_cost": 5,
        },
    },
}

# backtest and analysis
with R.start(experiment_name=EXP_NAME, recorder_id=rid, resume=True):
    # signal-based analysis
    rec2 = R.get_recorder()
    sar2 = SigAnaRecord(rec,ana_long_short=True)
    sar2.generate()

    #  portfolio-based analysis: backtest
    par2 = PortAnaRecord(rec, port_analysis_config2, "day")
    par2.generate()

[33612:MainThread](2023-11-21 11:31:44,240) INFO - qlib.workflow - [exp.py:258] - Experiment 466892454790696298 starts running ...
[33612:MainThread](2023-11-21 11:31:44,250) INFO - qlib.workflow - [recorder.py:341] - Recorder 05eee8f1acc94d1280152e884d344dad starts running under Experiment 466892454790696298 ...
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py", line 297, in log_param
    self.store.log_param(run_id, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 949, in log_param
    self._log_run_param(run_info, param)
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 955, in _log_run_param
    self._validate_new_param_value(
  File "/opt/conda/envs/qlib/lib/python3.8/site-packages/mlflow/store/tracking/file_store.py", line 975, in _validate_new_param_value
 

{'IC': 0.01894293685644901,
 'ICIR': 0.13929638116337453,
 'Long-Avg Ann Return': 0.42262878036126494,
 'Long-Avg Ann Sharpe': 2.1398666055831317,
 'Long-Short Ann Return': 0.10887369003428536,
 'Long-Short Ann Sharpe': 1.8776074506257552,
 'Rank IC': 0.04259207242290374,
 'Rank ICIR': 0.3089255517858425}


[33612:MainThread](2023-11-21 11:31:54,614) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[33612:MainThread](2023-11-21 11:31:54,615) WARNING - qlib.online operator - [exchange.py:219] - $close field data contains nan.
[33612:MainThread](2023-11-21 11:31:54,619) WARNING - qlib.online operator - [exchange.py:226] - factor.day.bin file not exists or factor contains `nan`. Order using adjusted_price.
[33612:MainThread](2023-11-21 11:31:54,620) WARNING - qlib.online operator - [exchange.py:228] - trade unit 100 is not supported in adjusted_price mode.
[33612:MainThread](2023-11-21 11:31:56,053) WARNING - qlib.BaseExecutor - [executor.py:121] - `common_infra` is not set for <qlib.backtest.executor.SimulatorExecutor object at 0x7f5ed58323a0>
backtest loop:   0%|          | 0/244 [00:00<?, ?it/s]/workspaces/Portfolio-Leveraging-Differentiable-Layer/qlib/qlib/utils/index_data.py:482: RuntimeWarning: Mean of empty slice
  return np.nanmean(self.data)
backt

'The following are analysis results of benchmark return(1day).'
                       risk
mean               0.001341
std                0.012507
annualized_return  0.319108
information_ratio  1.653894
max_drawdown      -0.140285
'The following are analysis results of the excess return without cost(1day).'
                       risk
mean               0.000339
std                0.004497
annualized_return  0.080709
information_ratio  1.163375
max_drawdown      -0.049059
'The following are analysis results of the excess return with cost(1day).'
                       risk
mean              -0.000823
std                0.004503
annualized_return -0.195838
information_ratio -2.818832
max_drawdown      -0.236795
'The following are analysis results of indicators(1day).'
     value
ffr    1.0
pa     0.0
pos    0.0
